This is the code base used for evaluating the performance of weakly labeled data augmentation in detecting COVID-19 disease manifestations in CXRs

In [ ]:
#load libraries
from keras import backend as K
K.clear_session()

from __future__ import print_function

from keras.models import Model, Input
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.layers import Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Concatenate
from keras.layers import SeparableConv2D
from scipy.ndimage.interpolation import zoom
import numpy as np
from keras.backend import tensorflow_backend
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from keras.layers import ZeroPadding2D, GlobalAveragePooling2D
import time
from scipy import interp
import cv2
import imutils
import pickle
import struct
import shutil
import numpy as np
import zlib
import pandas as pd
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam, SGD
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from keras.callbacks import CSVLogger
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt
import scikitplot as skplt
import itertools
from itertools import cycle
from sklearn.utils import class_weight
from keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pydicom
import numpy as np
import cv2
from matplotlib import pyplot as plt
import csv
import random
from shutil import copyfile
from tensorflow.python.framework import ops
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras import backend as K
from keras import applications
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
from matplotlib import pyplot as plt
from glob import glob
import skimage.io as io
import skimage.transform as trans
from PIL import Image
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import backend as keras
try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass

In [ ]:
#get current working directory
os.getcwd()

In [ ]:
#function to convert dicom to png

def dcm2png(input_dir: str, output_dir: str):
    """
    Becareful all output images are gray image with 8 bit
    :param input_dir: dcm file directory
    :param output_dir: save directory
    """
    if not os.path.isdir(input_dir):
        raise ValueError("Input dir is not found!")

    if not os.path.isdir(output_dir):
        raise ValueError("Out dir is not found!")

    img_list = [f for f in os.listdir(input_dir)
                if f.split('.')[-1] == 'dcm' or f.split('.')[-1] == 'jpeg'] 
    for n, f in enumerate(img_list):
        
        if f.split(".")[-1] == "dcm":
            dcm_file = input_dir + f
            ds = pydicom.dcmread(dcm_file)
            pixel_array_numpy = ds.pixel_array
            pixel_array_numpy = cv2.normalize(pixel_array_numpy,
                                              None,
                                              alpha=0,
                                              beta=255,
                                              norm_type=cv2.NORM_MINMAX,
                                              dtype=cv2.CV_8UC1)
            pixel_array_numpy = cv2.resize(pixel_array_numpy,
                                           (224,224))
            img_file = output_dir + f.replace('.dcm', '.png')
            cv2.imwrite(img_file, pixel_array_numpy)

        else:
            
            if f.split(".")[-1] == "jpeg":
                image_file = input_dir + f
                pixel_array_numpy = cv2.imread(image_file)
                pixel_array_numpy = cv2.cvtColor(pixel_array_numpy, cv2.COLOR_BGR2GRAY)
                pixel_array_numpy = cv2.resize(pixel_array_numpy,
                                               (224,224))
                image_file = output_dir + f.replace('.jpeg', '.png')
                cv2.imwrite(image_file, pixel_array_numpy)
                
                if n % 50 == 0:
                    print('{} image processed'.format(n))

#usage
#dcm2png("cxr_pneumonia_GT/test/PNEUMONIA/", "ped_pneumonia_256/test/PNEUMONIA/")


Begin U-net based semantic segmentation to genrate lung masks for the input CXRs

In [ ]:
#define different loss functions

def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#another implementation of dice coefficient from https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + binary_crossentropy(y_true, y_pred)

def bce_jac_loss(y_true, y_pred):
    return jaccard_distance(y_true, y_pred, smooth=100) + binary_crossentropy(y_true, y_pred)
   
def iou(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

    
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)


def threshold_binarize(x, threshold=0.5):
    ge = tf.greater_equal(x, tf.constant(threshold))
    y = tf.where(ge, x=tf.ones_like(x), y=tf.zeros_like(x))
    return y


def iou_thresholded(y_true, y_pred, threshold=0.5, smooth=1.):
    y_pred = threshold_binarize(y_pred, threshold)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

Define the Dilated UNET model with dilation rate 2 and dropout of 0.2.

In [ ]:
def dilatedunet(pretrained_weights = None,input_size = (224,224,1)): #image and masks of size 224x224x1
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.2)(conv4)  #empirically determine the best value
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.2)(conv5) 

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', 
                 dilation_rate=2, kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = Add()([drop4,up6])
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', 
                 dilation_rate=2, kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = Add()([conv3,up7])
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', 
                 dilation_rate=2, kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = Add()([conv2,up8])
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', 
                 dilation_rate=2, kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = Add()([conv1,up9])
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', dilation_rate=2, kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr=1e-4), loss=[bce_dice_loss], metrics=[iou, iou_thresholded,'accuracy']) #choose the best loss function
    
    model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model


Define the data format and the functions to train and test with the data using image generators. Make sure to use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same. If you want to visualize the results of generator, set save_to_dir = "your path"

In [ ]:
#create a color dictionary
A = [128,128,128]
B = [128,0,0]
C = [192,192,128]
D = [128,64,128]
E = [60,40,222]
F = [128,128,0]
G = [192,128,128]
H = [64,64,128]
I = [64,0,128]
J = [64,64,0]
K = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([A, B, C, D, E, F, G, H, I, J, K, Unlabelled])

def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],
                                        new_mask.shape[1]*new_mask.shape[2],
                                        new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)


def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (224, 224),seed = 1): 

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)
        
def valGenerator(batch_size,val_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (224, 224),seed = 1): 

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        val_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        val_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    val_generator = zip(image_generator, mask_generator)
    for (img,mask) in val_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

def testGenerator(test_path,target_size = (224,224),flag_multi_class = False,as_gray = True): 
    for filename in os.listdir(test_path):
        img = io.imread(os.path.join(test_path,filename),as_gray = as_gray) 
        img = img / 255.
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img


def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255


def saveResult(save_path,npyfile,test_path, flag_multi_class = False,num_class = 2):
    file_names = os.listdir(test_path)
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,file_names[i]),img)


In [ ]:
#train the dilated UNET model

data_gen_args = dict(rotation_range=10.,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=5,
                    zoom_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest') 
myGene = trainGenerator(2,'C:/Users/data/train',
                        'image','label',data_gen_args,save_to_dir = None) #batch size of 2 used here


model = dilatedunet()

callbacks = [EarlyStopping(monitor='loss', patience=15, verbose=1, min_delta=1e-4,
                           mode='min'),
             ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1,
                               epsilon=1e-4, mode='min'),
             ModelCheckpoint(monitor='loss', 
                             filepath='C:/Users/trained_model/dilatedunet.hdf5', 
                             save_best_only=True,
                             mode='min', verbose = 1)]
model.fit_generator(generator=myGene,steps_per_epoch=217, epochs=200, callbacks=callbacks,
                    verbose=1) #steps_per_epoch=training samples/batchsize + 1 if not absolutely divisible

In [ ]:
test_path = "C:/Users/data/test" 
save_path = "C:/Users/data/membrane/result" 

data_gen_args = dict(rotation_range=10.,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=5,
                    zoom_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest') 

testGene = testGenerator(test_path)
model = dilatedunet()
model.load_weights("C:/Users/trained_model/dilatedunet.hdf5")
results = model.predict_generator(testGene,135,verbose=1, workers=1, use_multiprocessing=False) 
#steps per epoch is the no. of samples in test image.
saveResult(save_path, results, test_path)

#postprocessing with the mask and image: This script helps to postprocess the images with the mask generated through the UNET and relax the boundaries by 5% on top, bottom, left, and right, and store the bounding box cordinates to a csv file. The cropped bounding box images are stored to a directory.

In [ ]:
#custom function to generate bounding boxes
def generate_bounding_box(image_dir: str, #containing images
                          mask_dir: str, #containing masks, images have same name as original images
                          dest_csv: str, #CSV file to write the bounding box coordinates
                          crop_save_dir: str): #save the cropped bounding box images
    """
    the orginal images are resized to 224x224
    the output crops are resized to 224x224
    """
    if not os.path.isdir(mask_dir):
        raise ValueError("mask_dir not existed")

    case_list = [f for f in os.listdir(mask_dir) if f.split(".")[-1] == 'png'] #all mask images are png files

    with open(dest_csv, 'w', newline='') as f:
        csv_writer = csv.writer(f)

        for j, case_name in enumerate(case_list):
            mask = cv2.imread(mask_dir + case_name)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            image = cv2.imread(image_dir + case_name, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (224, 224)) #original images are resized to 224x224
            if mask is None or image is None:
                raise ValueError("The image can not be read: " + case_name)

            reduce_col = np.sum(mask, axis=1)
            reduce_row = np.sum(mask, axis=0)
            # many 0s add up to none zero, we need to threshold it
            reduce_col = (reduce_col >= 255)*reduce_col
            reduce_row = (reduce_row >= 255)*reduce_row
            first_none_zero = None
            last_none_zero = None

            last = 0
            for i in range(reduce_col.shape[0]):
                current = reduce_col[i]
                if last == 0 and current != 0 and first_none_zero is None:
                    first_none_zero = i

                if current != 0:
                    last_none_zero = i

                last = reduce_col[i]

            up = first_none_zero
            down = last_none_zero

            first_none_zero = None
            last_none_zero = None
            last = 0
            for i in range(reduce_row.shape[0]):
                current = reduce_row[i]
                if last == 0 and current != 0 and first_none_zero is None:
                    first_none_zero = i

                if current != 0:
                    last_none_zero = i

                last = reduce_row[i]

            left = first_none_zero
            right = last_none_zero

            if up is None or down is None or left is None or right is None:
                raise ValueError("The border is not found: " + case_name)
            
            # new coordinates for image which is 1 times of mask, mask images are 224x224, 
            #so need to multiply 1 times to get 224x224, and relaxing the borders by 5% on all directions
            up_down_loose = int(1 * (down - up + 1) * 0.05)
            image_up = 1 * up - up_down_loose
            if image_up < 0:
                image_up = 0
            image_down = 1*(down+1)+up_down_loose
            if image_down > image.shape[0] + 1:
                image_down = image.shape[0]

            left_right_loose = int(1 * (right - left) * 0.05)
            image_left = 1 * left - left_right_loose
            if image_left < 0:
                image_left = 0
            image_right = 1*(right + 1)+left_right_loose
            if image_right > image.shape[1] + 1:
                image_right = image.shape[1]

            crop = image[image_up: image_down, image_left: image_right]
            crop = cv2.resize(crop, (224,224)) #the cropped image is resized to 224x224

            cv2.imwrite(crop_save_dir + case_name, crop) # cropped images saved to crop directory

            # write new csv
            crop_width = image_right - image_left + 1
            crop_height = image_down - image_up + 1

            csv_writer.writerow([case_name,
                                 image_left,
                                 image_up,
                                 crop_width,
                                 crop_height]) #writes xmin, ymin, width, and height

            if j % 50 == 0:
                print(j, " images are processed!")

#train-normal
generate_bounding_box("C:/Users/data/test/",
                      "C:/Users/result/mask_1/",
                      'C:/Users/result/bounding_box.csv',
                      "C:/Users/result/cropped1/")

Considering the case if the CXR images have ground truth ROI annotations, we need to downscale and store these values after bounding box cropping. In this regard, we can use the following custom function.

In [ ]:
def generate_bounding_box(image_dir: str,
                          mask_dir: str,
                          orgin_csv: str,
                          dest_csv: str,
                          crop_save_dir: str,
                          show_results: bool=False):
    if not os.path.isdir(mask_dir):
        raise ValueError("mask_dir not existed")

    csv_file_data = []
    with open(orgin_csv, 'r') as csvFile:
        reader = csv.reader(csvFile)
        for row in reader:
            csv_file_data.append(row)

    with open(dest_csv, 'w', newline='') as f:
        csv_writer = csv.writer(f)

        for j, row in enumerate(csv_file_data[1:]):
            case_name = row[0] + '.png' #all mask files are png file type
            mask = cv2.imread(mask_dir + case_name)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            image = cv2.imread(image_dir + case_name, cv2.COLOR_BGR2GRAY)
            #original images are resized to 256x256, comment this if you want to keep the original image resolution
            image = cv2.resize(image, (256,256)) 
            if mask is None or image is None:
                raise ValueError("The image can not be read: " + case_name)
            
            reduce_col = np.sum(mask, axis=1)
            reduce_row = np.sum(mask, axis=0)
            # many 0s add up to none zero, we need to threshold it
            reduce_col = (reduce_col >= 255)*reduce_col
            reduce_row = (reduce_row >= 255)*reduce_row
            first_none_zero = None
            last_none_zero = None

            last = 0
            for i in range(reduce_col.shape[0]):
                current = reduce_col[i]
                if last == 0 and current != 0 and first_none_zero is None:
                    first_none_zero = i

                if current != 0:
                    last_none_zero = i

                last = reduce_col[i]

            up = first_none_zero
            down = last_none_zero

            first_none_zero = None
            last_none_zero = None
            last = 0
            for i in range(reduce_row.shape[0]):
                current = reduce_row[i]
                if last == 0 and current != 0 and first_none_zero is None:
                    first_none_zero = i

                if current != 0:
                    last_none_zero = i

                last = reduce_row[i]

            left = first_none_zero
            right = last_none_zero
            
            if up is None or down is None or left is None or right is None:
                raise ValueError("The border is not found: " + case_name)
            
            # new coordinates for image which is 1 times of mask, mask images are 256x256, 
            #need to multiply 1 times to get 256x256, and relaxing the borders by 5% on all directions
            
            loose = int(1 * (down - up + 1) * 0.05) 
            # for example, if the original image resolution is 
            #1024x1024, new coordinates for image is 4 times of mask (256x256), 
            #need to multiply 4 times to 1024x1024, and relaxing the borders by 5% on all directions, 
            #i.e. int(4 * (down - up + 1) * 0.05)
            image_up = 1 * up - loose #4
            if image_up < 0:
                image_up = 0
            image_down = 1*(down+1)+loose #4
            if image_down > image.shape[0] + 1:
                image_down = image.shape[0]

            loose2 = int(1 * (right - left + 1) * 0.05) #4
            image_left = 1 * left - loose2 #4
            if image_left < 0:
                image_left = 0
            image_right = 1*(right+1)+loose2 #4
            if image_right > image.shape[1] + 1:
                image_right = image.shape[1]

            crop = image[image_up: image_down, image_left: image_right]
            crop = cv2.resize(crop, (256,256)) #1024, 1024 before

            # store images in normal or abnormal folder
            if row[6] != 'Normal':
                subfolder = 'abnormal/'
            else:
                subfolder = 'normal/'
            cv2.imwrite(crop_save_dir + subfolder + case_name, crop)

            # write new csv
            crop_width = image_right - image_left + 1
            crop_height = image_down - image_up + 1

            if row[6] == "Lung Opacity":

                y_scale_change = 256 / crop_height #1024 before
                x_scale_change = 256 / crop_width # 1024 before, image size was 1024 by 1024

                bbox_y = int(float(row[2])) 
                #new_y = int((bbox_y - image_up) * y_scale_change)
                new_y = int((bbox_y/4 - image_up) * y_scale_change) #since resized to 256 from 1024, scale by 1/4

                bbox_x = int(float(row[1]))
                #new_x = int((bbox_x - image_left) * x_scale_change)
                new_x = int((bbox_x/4 - image_left) * x_scale_change)  #since resized to 256 from 1024, scale by 1/4

                bbox_width = int(float(row[3]))
                bbox_height = int(float(row[4]))
                #new_width = int(bbox_width/4 * x_scale_change)
                #new_height = int(bbox_height/4 * y_scale_change)
                new_width = int(bbox_width/4 * x_scale_change)  #since resized to 256 from 1024, scale by 1/4
                new_height = int(bbox_height/4 * y_scale_change) #since resized to 256 from 1024, scale by 1/4

                csv_writer.writerow([case_name,
                                     image_left,
                                     image_up,
                                     crop_width,
                                     crop_height,
                                     new_x,
                                     new_y,
                                     new_width,
                                     new_height,
                                     row[6]])

                else:
                csv_writer.writerow([case_name,
                                     image_left,
                                     image_up,
                                     crop_width,
                                     crop_height,
                                     "",
                                     "",
                                     "",
                                     "",
                                     row[6]])
            """
            if show_results:
                
                # image distribution
                plt.hist(mask.ravel(), 256, [0, 256])
                mask = mask / 255
                # Bounding box for image and mask
                mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
                rect_mask = cv2.rectangle(mask, (left, up), (right, down), (0, 255, 0), 2)
                rect_image = cv2.rectangle(image, (4 * left, 4 * up), (4 * right, 4 * down), (0, 255, 0), 2)
                cv2.imshow("crop", crop)
                cv2.imshow("mask", rect_mask)
                cv2.imshow("image", rect_image)
            """
            if j % 100 == 0:
                print(j, " images are processed!")


generate_bounding_box("C:/Users/dcm_2_image/",
                      "C:/Users/dcm_2_image_mask/",
                      'C:/Users/stage_2_train_labels.csv',
                      'C:/Users/bounding_box_rsna_256.csv',
                      "C:/Users/rajaramans2/codes/crops/",
                      show_results=False)

Now tht we have preprocessed the orignal CXRs and cropped them to a size of the bounding box with 224x224 pixel resoultion and grayscale images, we can begin training the models. Initially all models are trained and tested on the pediatric CXR collection to classify them into bacterial and viral pneumonia categories. The best model with these pretrained weights is used to predict the Montreal and Twitter COVID-19 CXR collections as belonging to bacterial or viral pnuemonia cateogry and the results are tabulated.

In [ ]:
#%% custom function to plot confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#%% path to input data
train_data_dir = '.../train'
test_data_dir = '.../test'
img_width = 224
img_height = 224
channel = 3
epochs = 32
batch_size = 8 #vary this parameter depending on your GPU capacity
num_classes= 2 #[bacterial, viral]
input_shape = (img_width, img_height, channel)
model_input = Input(shape=input_shape)
print(model_input) 

In [ ]:
#%% declared data generators, allocate 20% of the training data for validation

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_data_dir, 
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=True,
    seed=13,
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=True,
    seed=13,
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(val_generator.filenames)
nb_test_samples = len(test_generator.filenames)

#check the class indices
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

#true labels
Y_test=test_generator.classes
print(Y_test.shape)

#convert test labels to categorical
Y_test1=to_categorical(Y_test, num_classes=num_classes, dtype='float32')
print(Y_test1.shape)

we use a wide residual custom CNN for this task. Model architecture taken from https://github.com/titu1994/Wide-Residual-Networks

In [ ]:
weight_decay = 0.0005
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', 
                      strides=strides, kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', 
                         strides=strides, kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m

def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, 
                           epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    ip = Input(shape=input_dim)
    x = initial_conv(ip)
    nb_conv = 4
    x = expand_conv(x, 16, k)
    nb_conv += 2
    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2
    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2
    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2
    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2
    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2
    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)
    x = Dense(nb_classes, W_regularizer=l2(weight_decay), activation='softmax')(x)
    model = Model(ip, x, name = 'wide-residual-cnn')
    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

In [ ]:
#initiate custom WRN model with a depth of 4 and width of 10, and dropout of 0.3
model = create_wide_residual_network(input_shape, nb_classes=2, 
                                            N=4, k=10, dropout=0.3, verbose=1) 
model.summary()

Declare the pretrained model architecture: models used in this study: VGG-16, Inception-V3, Xception, DenseNet-121, and NasNet-mobile. The truncated models are added with a global average pooling and final dense layer and retrained from the scratch. Here, we show the training process with the VGG-16 model. It has to be replaced with the other pretrained models when trained.

In [ ]:
feature_model = applications.VGG16(weights='imagenet', 
                                   include_top=False,
                                   input_shape=(img_width,img_height,channel))
feature_model = Model(inputs=feature_model.input,
                    outputs=feature_model.get_layer('block5_conv3').output)
feature1_model.summary()

#addind the top layers
x = feature1_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax', 
                    name='predictions')(x)
model = Model(inputs=feature_model.input, 
              outputs=predictions, name='vgg16_base_model')
model.summary()

#enumerate and print layer names
for i, layer in enumerate(model.layers):
   print(i, layer.name) 

In [ ]:
#%% compute class weights
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
print(class_weights)

In [ ]:
#%% fix optimizer and start training the model
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.95, nesterov=True) #optimize to your requirements
#compile the model
model.compile(optimizer=sgd,              
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#reset the generators first or it will give weird results
train_generator.reset()
val_generator.reset()

# start training
start = time.time()
filepath = 'weights/' + model.name + '.{epoch:02d}-{val_acc:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_weights_only=False, 
                             save_best_only=True, mode='min', period=1)
earlyStopping = EarlyStopping(monitor='val_loss', 
                               patience=10, verbose=1, mode='min')
tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=batch_size)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5,
                              verbose=1, mode='min', min_lr=0.00001)
callbacks_list = [checkpoint, tensor_board, earlyStopping, reduce_lr]

custom_vgg16_history = model.fit_generator(
      train_generator,
      steps_per_epoch=nb_train_samples // batch_size + 1, #see if not absolutely divisble by batch size
      epochs=epochs,
      validation_data=val_generator,
      callbacks=callbacks_list,
      class_weight = class_weights,
      validation_steps=nb_validation_samples // batch_size + 1, 
      verbose=1)

#print the total time taken for training
print(time.time()-start)

In [ ]:
#%% plot performance
N = epochs
plt.style.use("ggplot")
plt.figure(figsize=(20,10), dpi=300)
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["loss"], 'orange', label="train_loss")
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["val_loss"], 'red', label="val_loss")
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["acc"], 'blue', label="train_acc")
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["val_acc"], 'green', label="val_acc")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower right")
plt.savefig("vgg16_performance.png")

In [ ]:
#%% evaluate with the best stored model weights
model.load_weights('weights/vgg16_base_model.14-0.9154.h5') #change this to your path and model weights
model.summary()
#compile the model
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.95, nesterov=True) #optimize to your requirements
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#predict
test_generator.reset()
custom_vgg16_y_pred = model.predict_generator(test_generator, 
                                              nb_test_samples/batch_size, workers=1)
#true labels
Y_test=test_generator.classes

#print the shape of y_pred and Y_test
print(custom_vgg16_y_pred.shape)
print(Y_test.shape)

#measure accuracy
custom_vgg16_model_accuracy=accuracy_score(Y_test,custom_vgg16_y_pred.argmax(axis=-1))
print('The accuracy of custom VGG16 model is: ', custom_vgg16_model_accuracy)

In [ ]:
#%% #print classification report

target_names = ['class 0(bacterial)', 'class 1(viral)'] #from the generator.class_indices
print(classification_report(Y_test,
                            custom_vgg16_y_pred.argmax(axis=-1),
                            target_names=target_names, digits=4))
# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test, custom_vgg16_y_pred.argmax(axis=-1))
np.set_printoptions(precision=4)
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for custom VGG16 model without normalization')
plt.show()

In [ ]:
#store predictions: 
#predicted_class_indices has the predicted labels, 0 for bacterial and 1 for viral
predicted_class_indices=np.argmax(custom_vgg16_y_pred,axis=1)

#map the predicted labels with their unique ids 
#such as filenames to find out what you predicted for which image.
labels = (train_generator.class_indices) # 0 for bacterial and 1 for viral
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices] #displays as string

#Finally, save the results to a CSV file.
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("predictions_base_vgg16.csv",index=False)

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test1[:, i], custom_vgg16_y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
  
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_test1.ravel(), custom_vgg16_y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
lw = 2
fig=plt.figure(figsize=(15,10), dpi=70)
ax = fig.add_subplot(1, 1, 1)
# Major ticks every 0.05, minor ticks every 0.05
major_ticks = np.arange(0.0, 1.0, 0.05)
minor_ticks = np.arange(0.0, 1.0, 0.05)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')

#plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.4f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('multi-class ROC curves')
plt.legend(loc="lower right")
plt.show()

Test the trained model with covid-19 cxr collection from Montreal and Twitter and record the baseline performance: We will also test with the data from other datasets including RSNA, CheXpert, and NIH and weakly label them to bacterial and viral cases and store them separately.

In [ ]:
#load test data directory and declare test generators
#test_data_dir = 'datasets/cropped_ieee_covid19_04152020'
#test_data_dir = 'datasets/cropped_twitter_covid19'
#test_data_dir = 'datasets/chexpert_dataset/chexpert_pneumonia'
test_data_dir = 'datasets/rsna_dataset/rsna_pneumonia'
#test_data_dir = 'datasets/nih_dataset/nih_pneumonia'

#declare test generators
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=1,
        class_mode=None,
        shuffle=False)

nb_test_samples = len(test_generator.filenames)

#check the class indices
print(test_generator.class_indices)

#true labels
Y_test=test_generator.classes
print(Y_test.shape)

#convert test labels to categorical
Y_test1=to_categorical(Y_test, num_classes=num_classes, dtype='float32')
print(Y_test1.shape)

In [ ]:
#%% load the best model and predict on the test data
model.load_weights('weights/vgg16_base_model.h5') # the best performing model from baseline
model.summary()
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.95, nesterov=True) #optimize to your requirements

#compile the model
model.compile(optimizer=sgd,              
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#predict
test_generator.reset()
print('-'*30)
print('Predicting on the test data...')
print('-'*30)
y_pred_base = model.predict_generator(test_generator,
                                 (nb_test_samples//1),
                                 verbose=1)
#print prediction shapes
print(y_pred_base.shape)
print(Y_test.shape)

#measure accuracy
model_accuracy=accuracy_score(Y_test,y_pred_base.argmax(axis=-1))
print('The accuracy of the model is: ', model_accuracy)

In [ ]:
#%% #print classification report
target_names = ['Bacterial_pneumonia', 'Viral_pneumonia']
print(classification_report(Y_test,y_pred_base.argmax(axis=-1),
                            target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,y_pred_base.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot normalized confusion matrix using scikit plot
skplt.metrics.plot_confusion_matrix(Y_test,y_pred_base.argmax(axis=-1),
                                    normalize=False, x_tick_rotation=45, figsize=(15,10),
                                    title_fontsize='large', text_fontsize='medium')
plt.show()

# Plot non-normalized confusion matrix using scikit learn
plt.figure(figsize=(10,10), dpi=70)
plot_confusion_matrix(cnf_matrix, classes=target_names)
plt.show()

In [ ]:
#predicted_class_indices has the predicted labels, 0 for bacterial and 1 for viral
predicted_class_indices=np.argmax(y_pred_base,axis=1)
#map the predicted labels with their unique ids 
#such as filenames to find out what you predicted for which image.
labels = (train_generator.class_indices) # 0 for bacterial and 1 for viral
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices] #displays as string

#Finally, save the results to a CSV file.
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("rsna_vgg16.csv",index=False)

Using the CSV file we will copy the loosely labled CXR images as bacterial or viral pneumonia into separate folders. Here the process is shown with the RSNA pneumonia collection loosely labled into bacterial and viral pneumonia

In [ ]:
src = "C:\\Users\\datasets\\rsna_pneumonia\\"
dst = "C:\\Users\\rsna_bacterial_pneumonia\\"
dst1 = "C:\\Users\\rsna_viral_pneumonia\\"
df = pd.read_csv('rsna_vgg16.csv') 
df_names = df['Filename'].values
df_class = df['Predictions'].values
for i in range(len(df_class)):
    src_path = os.path.join(src, df_names[i])
    if df_class[i] == 'viral':
        #dst_path = os.path.join(dst1,df_names[i].split(os.sep)[-1])
        dst_path = os.path.join(dst1,df_names[i])
        shutil.copy(src_path, dst_path)
    else:
        #dst_path = os.path.join(dst,df_names[i].split(os.sep)[-1])
        dst_path = os.path.join(dst,df_names[i])
        shutil.copy(src_path, dst_path)

Now augment the baseline train data with the weakly labeled bacterial and viral CXR images from each of the datasets and also on various combinations as mentioned:
baseline, baseline+rsna, baseline+nih, baseline+chexpert, baseline+rsna+chexpert, baseline+rsna+nih, baseline+nih+chexpert,
baseline+rsna+nih+chexpert
The test data will be the baseline test data, Montreal covid19, or Twitter covid-19. we will compare the performance of the baseline model on these three different test data, with the performance achieved by training the models with the aforementioned eight combinations of weak label augmentation with an aim to improve performance.

In [ ]:
#declare the path to data
train_data_dir = 'datasets/base_nih/train'
#train_data_dir = 'datasets/base_rsna/train'
#train_data_dir = 'datasets/base_chexpert/train'
#train_data_dir = 'datasets/base_nih_rsna/train'
#train_data_dir = 'datasets/base_nih_chexpert/train'
#train_data_dir = 'datasets/base_rsna_chexpert/train'
#train_data_dir = 'datasets/base_nih_rsna_chexpert/train'

#the test directory will be one of the following during varius evaluations:
test_data_dir = 'datasets/cropped_ieee_covid19'
#test_data_dir = 'datasets/cropped_twitter_covid19'
#test_data_dir = 'datasets/cell_pneumonia_224_8/test'

img_width = 224
img_height = 224
channel = 3
epochs = 32
batch_size = 8 #vary this parameter depending on your GPU capacity
input_shape = (img_width, img_height, channel)
model_input = Input(shape=input_shape)
print(model_input) 

In [ ]:
#%% declared data generators, allocate 20% of the training data for validation
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_data_dir, 
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset='training'
)
val_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset='validation'
)
test_datagen = ImageDataGenerator(rescale=1./255)

"""test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False) #should be used when testing with ground truth cell pneumonia dataset
"""
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=1,
        class_mode=None,
        shuffle=False)

nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(val_generator.filenames)
nb_test_samples = len(test_generator.filenames)

#check the class indices
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

#true labels
Y_test=test_generator.classes
print(Y_test.shape)

#convert test labels to categorical
Y_test1=to_categorical(Y_test, num_classes=num_classes, dtype='float32')
print(Y_test1.shape)

use the best performing baseline model vgg-16 in this study, and train it on the augmented train data and evaluate the performance with the different test data used in this study. i.e. ieee covid-19, twitter covid-19 and beseline test data. Repeat the process for each of the augmented combination aforementioned. 

In [ ]:
feature1_model = applications.VGG16(weights='imagenet', 
                                   include_top=False,
                                   input_shape=(img_width,img_height,channel))
feature1_model = Model(inputs=feature1_model.input,
                    outputs=feature1_model.get_layer('block5_conv3').output)

feature1_model.summary()
#addind the top layers
x = feature1_model.output
x = GlobalAveragePooling2D() (x)
predictions = Dense(num_classes, activation='softmax', 
                    name='predictions')(x)
model = Model(inputs=feature1_model.input, 
              outputs=predictions, name='vgg16_augmented_base_nih_model')
model.summary()

#enumerate and print layer names
for i, layer in enumerate(model.layers):
   print(i, layer.name) 

In [ ]:
# compute class weights
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
print(class_weights)

In [ ]:
#fix optimizer and start training the model
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.95, nesterov=True) #optimize to your requirements
#compile the model
model.compile(optimizer=sgd,              
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#reset the generators 
train_generator.reset()
val_generator.reset()

# start training
start = time.time()
filepath = 'weights/' + model.name + '.{epoch:02d}-{val_acc:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_weights_only=False, 
                             save_best_only=True, mode='min', period=1)
earlyStopping = EarlyStopping(monitor='val_loss', 
                               patience=10, verbose=1, mode='min')
tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=batch_size)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5,
                              verbose=1, mode='min', min_lr=0.00001)
callbacks_list = [checkpoint, tensor_board, earlyStopping, reduce_lr]

custom_vgg16_history = model.fit_generator(
      train_generator,
      steps_per_epoch=nb_train_samples // batch_size + 1,
      epochs=epochs,
      validation_data=val_generator,
      callbacks=callbacks_list,
      class_weight = class_weights,
      validation_steps=nb_validation_samples // batch_size + 1, 
      verbose=1)

#print the total time taken for training
print(time.time()-start)

In [ ]:
#plot performance
N = epochs
plt.style.use("ggplot")
plt.figure(figsize=(20,10), dpi=300)
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["loss"], 'orange', label="train_loss")
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["val_loss"], 'red', label="val_loss")
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["acc"], 'blue', label="train_acc")
plt.plot(np.arange(1, N+1), 
         custom_vgg16_history.history["val_acc"], 'green', label="val_acc")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower right")
plt.savefig("vgg16_augmented_performance.png")

In [ ]:
#%% evaluate with the best stored model weights
model.load_weights('weights/vgg16_augmented_model.14-0.9347.h5') #change this to your path and model weights
model.summary()
#compile the model
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.95, nesterov=True) #optimize to your requirements
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#%% when evaluating with the baseline hold out test data
#test_generator.reset()
#y_pred_val = model.predict_generator(val_generator, 
#                                                   (nb_validation_samples/batch_size),
#                                                    verbose=1)
##measure accuracy
#vgg16_val_accuracy=accuracy_score(Y_test,y_pred_val.argmax(axis=-1))
#print('The accuracy of custom VGG16 model is: ', vgg16_val_accuracy)

#predict with Montreal or Twitter COVID-19 test data
print('-'*30)
print('Predicting on the test data...')
print('-'*30)
test_generator.reset()
y_pred_base = model.predict_generator(test_generator,
                                 (nb_test_samples//1),
                                 verbose=1)
#print prediction shapes
print(y_pred_base.shape)
print(Y_test.shape)

#measure accuracy
model_accuracy=accuracy_score(Y_test,y_pred_base.argmax(axis=-1))
print('The accuracy of the model is: ', model_accuracy)

In [ ]:
#print classification report
target_names = ['Bacterial_pneumonia', 'Viral_pneumonia']
print(classification_report(Y_test,y_pred_base.argmax(axis=-1),
                            target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,y_pred_base.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot normalized confusion matrix using scikit plot
skplt.metrics.plot_confusion_matrix(Y_test,y_pred_base.argmax(axis=-1),
                                    normalize=False, x_tick_rotation=45, figsize=(15,10),
                                    title_fontsize='large', text_fontsize='medium')
plt.show()

# Plot non-normalized confusion matrix using scikit learn
plt.figure(figsize=(10,10), dpi=70)
plot_confusion_matrix(cnf_matrix, classes=target_names)
plt.show()

In [ ]:
#predicted_class_indices has the predicted labels
predicted_class_indices=np.argmax(y_pred_base,axis=1)

#map the predicted labels with their unique ids 
#such as filenames to find out what you predicted for which image.

labels = (train_generator.class_indices) 
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices] #displays as string

#Finally, save the results to a CSV file.
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("ieee_base_nih_chexpert_vgg16.csv",index=False)

Similarly, record the performance with each of the pooled train collections with respect to the different test data. The prediction performance with the pooled training data should be different from that with the baseline non-augmented train data.
we further improve the performance by performing an union of the pateint-IDs with viral pneumonia predictions from each of the predictions from the pooled training data. in that way we found that computing the union of the patient IDs with viral pneumonia predictions obtained by training the model with baseline and (baseline+chexpert) data gave promising performance than
other combinations for the IEEE and Twitter COVID-19 data collection. 

Visualization studies: To visualize the learned behavior of the baseline model on the baseline test data and unseen twitter and covid-19 collection

In [ ]:
#load model
model = load_model('vgg16_cell_pneumonia_detector.10-0.9282.h5')
model.summary()
#compile the model
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.95, nesterov=True) #optimize to your requirements
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#path to image to visualize
img_path = 'person78_bacteria_380.png'
img = image.load_img(img_path)

#preprocess the image
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255 

#predict on the image
preds = model.predict(x)[0]
print(preds)

In [ ]:
#begin visualization
covid_output = model.output[:, 0] #index 1 of the viral pneumonia class, 0 for baterial class; change as per image input

#Output feature map from the last convolutional layer
last_conv_layer = model.get_layer('block5_conv3')

#compute the Gradient of the expected class 
#with regard to the output feature map of block5_conv3 
#(or the deepst convolutional layer)
grads = K.gradients(covid_output, last_conv_layer.output)[0]

#Vector of shape (512,), where each entry is the mean intensity 
#of the gradient over a specific feature-map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

In [ ]:
#access the values of the quantities we defined: 
#pooled_grads and the output feature map of block5_conv3, given a sample image
iterate = K.function([model.input],[pooled_grads, last_conv_layer.output[0]])

#Values of these two quantities, as Numpy arrays, given the sample image
pooled_grads_value, conv_layer_output_value = iterate([x])

#Multiplies each channel in the feature-map array by 
#“how important this channel is” with regard to the expected class
for i in range(512): #number of filters in the deepest convolutional layer
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

#For visualization purposes, we normalize the heatmap between 0 and 1.
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)

In [ ]:
# another code peice taken from stack overflow for writng the png output with different DPIs.

def writePNGwithdpi(im, filename, dpi=(72,72)):
   """Save the image as PNG with embedded dpi"""

   # Encode as PNG into memory
   retval, buffer = cv2.imencode(".png", im)
   s = buffer.tostring()

   # Find start of IDAT chunk
   IDAToffset = s.find(b'IDAT') - 4
   pHYs = b'pHYs' + struct.pack('!IIc',int(dpi[0]/0.0254),int(dpi[1]/0.0254),b"\x01" ) 
   pHYs = struct.pack('!I',9) + pHYs + struct.pack('!I',zlib.crc32(pHYs))
   with open(filename, "wb") as out:
      out.write(buffer[0:IDAToffset])
      out.write(pHYs)
      out.write(buffer[IDAToffset:])

In [ ]:
#using OpenCV to generate an image that superimposes the original image on the heatmap obtained
img = cv2.imread(img_path)

#Resizes the heatmap to be the same size as the original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

#Converts the heatmap to RGB 
heatmap = np.uint8(255 * heatmap)

#Applies the heatmap to the original image
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.3 + img #0.4 here is a heatmap intensity factor.

#Saves the image to disk
#cv2.imwrite('covid_cam_vgg16_fourthimage.jpg', superimposed_img)

#if we have to increse the DPI and write to disk
writePNGwithdpi(superimposed_img, "img_gradcam_vis.png", (300,300))

In [ ]:
We can use LIME visualization to interpret the models behavior

In [ ]:
#load image
img = image.load_img(img_path)

#preprocess the image
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255 

#predict on the image
preds = model.predict(x)[0]

In [ ]:
#initialize the explainer
from lime import lime_image
from skimage.segmentation import mark_boundaries
explainer = lime_image.LimeImageExplainer()

The following are the parameters to modify:
hide_color- is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels. Here, we set it to 0 (means gray).
top_labels - if not None, ignore labels and produce explanations for the K labels with highest prediction probabilities, where K is this parameter.

In [ ]:
explanation = explainer.explain_instance(x[0], 
                                         model.predict, top_labels=1, #produces explanations for the top 1 labels
                                         hide_color=0, num_samples=42)
print(explanation.top_labels[0])

Wall time: more than 3 minutes in local windows system. 
Explanation: for the top-class. We can see the top 5 superpixels that are most positive towards the class with the rest of the image hidden

In [ ]:
temp, mask = explanation.get_image_and_mask(0, #change the respective class index
                                            positive_only=False, 
                                            num_features=5, hide_rest=False) 
plt.figure()
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.figure()
plt.imshow(x[0] / 2 + 0.5) #this increases te brightness of the image